# Quick Visualizations

This notebook allows selection of files with a dropdown box in the plotting interface to quickly look at scenes. It should would with EMIT CWC outputs, ECOSTRESS, or EMIT RFL files. Just define the directory.

In [1]:
import os
import glob
import earthaccess
import math
import numpy as np
import xarray as xr
from osgeo import gdal
import rasterio as rio
import rioxarray as rxr
from matplotlib import pyplot as plt
import panel as pn
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
import pandas as pd
import geopandas as gp
import sys
from modules.emit_tools import emit_xarray, ortho_xr

Change the `file_dir` to the directory with the scenes you want to preview.

In [16]:
# Define a Path to the directory of imagery
file_dir = "../../shared/2023-VITALS-Workshop-AGU/data/cropped/dangermond/ewt/"

In [17]:
# List netcdf4 files and tif files in a directory
file_list = [f'{file.split("/")[-1]}' for file in sorted(glob.glob(file_dir+"*.nc"))]+[f'{file.split("/")[-1]}' for file in sorted(glob.glob(file_dir+"*.tif"))]
file_list

['EMIT_L2A_RFL_001_20230219T202939_2305013_002_dangermond_cwc.tif',
 'EMIT_L2A_RFL_001_20230219T202939_2305013_002_dangermond_cwc_merged.tif',
 'EMIT_L2A_RFL_001_20230219T202951_2305013_003_dangermond_cwc.tif',
 'EMIT_L2A_RFL_001_20230401T203751_2309114_002_dangermond_cwc.tif',
 'EMIT_L2A_RFL_001_20230405T190311_2309513_002_dangermond_cwc.tif',
 'EMIT_L2A_RFL_001_20230422T195924_2311213_002_dangermond_cwc.tif',
 'EMIT_L2A_RFL_001_20230629T170449_2318011_002_dangermond_cwc.tif',
 'EMIT_L2A_RFL_001_20230923T232101_2326615_002_dangermond_cwc.tif',
 'EMIT_L2A_RFL_001_20231014T224006_2328715_002_dangermond_cwc.tif']

In [18]:
# Select Scene/File using a Widget
scene_sel = pn.widgets.Select(name='Scene', options=file_list, value = file_list[0]) 
@pn.depends(scene_sel)
def scene_browse(scene_sel):
    # Define Filepath to open using selection
    fp = f"{file_dir}{scene_sel}"
    # For CWC
    if fp.split('.')[-1] == 'tif':
        ds = rxr.open_rasterio(fp).squeeze('band',drop=True)
         # For CWC/EMIT Derived COGS
        if "time_coverage_start" in list(ds.attrs.keys()):
            plt_title=f"CWC (g/cm^2) {ds.time_coverage_start.split('+')[0]}"
        # For ECOSTRESS
        else:
            plt_title=f"ECOSTRESS {scene_sel.split('_')[-5:-3]}"
           
        map = ds.hvplot.image(x='x',y='y',geo=True, cmap='viridis',tiles='ESRI',
                 title=plt_title,
                 xlabel='Longitude',ylabel='Latitude', frame_width=500,
                 frame_height=500, fontscale=1.5, alpha = 0.7)
    # For EMIT NetCDF4 Files
    if fp.split('.')[-1] == 'nc':
        # Open Dataset
        ds = xr.open_dataset(fp, decode_coords='all')
        
        # Conditionals for Subset/Original Dataset and Orthorectification
        if "spatial_ref" in list(ds.variables.keys()):
            ortho = True
        else:
            ortho = False 
        if "bands" in ds.dims:
            ds = emit_xarray(fp)
        if 'wavelengths' not in ds.dims:
            ds = ds.sel(mask_bands='Aggregate Flag')
            plt_title = f"{ds.time_coverage_start.split('+')[0]} - {ds.mask_bands.values}"
        else:
            ds = ds.sel(wavelengths=850,method='nearest')
            plt_title = f"{ds.time_coverage_start.split('+')[0]} - {ds.wavelengths.values:.3f} nm"
        if ortho == False:
            ds = ortho_xr(ds)
        map = ds.hvplot.image(x='longitude',y='latitude',geo=True, cmap='viridis',tiles='ESRI',
                 title=plt_title,
                 xlabel='Longitude',ylabel='Latitude', frame_width=500,
                 frame_height=500, fontscale=1.5, alpha=0.7)
    return map 
pn.Row(pn.WidgetBox(scene_sel), scene_browse)

Row
    [0] WidgetBox
        [0] Select(name='Scene', options=['EMIT_L2A_RFL_001_2023021...], value='EMIT_L2A_RFL_001_20230219...)
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False)